### Change Log
* Version 1: Copied from original notebook
* Version 2: train_nn created here; added holdout set
* Version 3: Moved the creation of holdout set to training notebook
* Version 5: Cleaned for only test processing
* Version 6: Added min max scaler for NN dataset
* Version 7:
    - Prepare TabNet data inside this notebook
    - Design synthetic test set for debugging submission scoring error
* Version 8: Stock clustering (categorical column)
* Version 10: NN label encoding for embedding
* Version 12: Time clustering (categorical column)
* Version 13: Time clustering - only log transform positive columns
* Version 14: Added velocity features
* Version 15: Added Time Series AutoEncoding features
   

### To do list:
* Add skew and kurt and add to stock level aggregation
* Add velocity
* Wavelength and Amplitudes
* Time clustering?
* TS clustering by AE
* Prepare TabNet data inside this notebook
* Time features - only log-transform non-negative columns

In [1]:
TRAIN = False
TEST_MODE = 'test'
FE_PATH = '/kaggle/input/volatility-fe-output-version-15'
N_STOCK = 112
N_SYN_STOCK = 2
SEED = 1111

In [2]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np # linear algebra
import glob
import os
import gc
import datetime
import pickle

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib
pd.set_option('display.max_columns', None)

In [3]:
# df = pd.DataFrame({'a':[1,2,3,4,5], 'b':['a','a','b','b','c']})
# df.groupby('b')['a'].agg(lambda s:s.max()-s.min())

In [4]:
# data directory
data_dir = '../input/optiver-realized-volatility-prediction/'

# fill inf and nan of  a dataframe
def fill_inf_nan(df):
    df = df.replace([np.inf, -np.inf], np.nan) # replace Inf with NaN
    df = df.fillna(df.mean()) # replace NaN with mean
    df = df.fillna(0) # if there are still na, fill with zero
    return df

# function to load a book/trade train/test single stock file
def load_single_stock(stock_id, train_test, book_trade):
    path = f'/kaggle/input/optiver-realized-volatility-prediction/{book_trade}_{train_test}.parquet/stock_id={str(stock_id)}'
    filename = os.path.join(path, os.listdir(path)[0])
    return pd.read_parquet(filename)

# Function to calculate first WAP
def calc_wap1(df):
    return (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
def calc_wap2(df):
    return (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
def calc_wap3(df):
    return (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
def calc_wap4(df):
    return (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])

# Function to calculate the log of the return
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

# function to return range of a series
def min_max_range(s):
    return s.max() - s.min()

# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

# Function to preprocess book data (for each stock id)
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    df['wap3'] = calc_wap3(df)
    df['wap4'] = calc_wap4(df)
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(log_return)
    df['log_return4'] = df.groupby(['time_id'])['wap4'].apply(log_return)
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    # Moving Avereage features
    df['log_return1_sma'] = df.groupby('time_id')['log_return1'].apply(lambda s: s.rolling(window=50).mean())
    df['log_return1_sms'] = df.groupby(['time_id'])['log_return1'].apply(lambda s: s.rolling(window=50).std())
    df['log_return1_sma_diff'] = df.groupby('time_id')['log_return1'].apply(lambda s: s.rolling(window=20).mean()) - df.groupby('time_id')['log_return1'].apply(lambda s: s.rolling(window=50).mean())
    df['log_return1_sms_diff'] = df.groupby(['time_id'])['log_return1'].apply(lambda s: s.rolling(window=20).std()) - df.groupby(['time_id'])['log_return1'].apply(lambda s: s.rolling(window=50).std())

    # Dict for aggregations; These are the features that only exist in full seconds range (000-599)
    create_feature_dict = {
        'wap1': [np.sum, np.std],
        'wap2': [np.sum, np.std],
        'wap3': [np.sum, np.std],
        'wap4': [np.sum, np.std],
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
        'wap_balance': [np.sum, np.max],
        'price_spread':[np.sum, np.max],
        'price_spread2':[np.sum, np.max],
        'bid_spread':[np.sum, np.max],
        'ask_spread':[np.sum, np.max],
        'total_volume':[np.sum, np.max],
        'volume_imbalance':[np.sum, np.max],
        "bid_ask_spread":[np.sum,  np.max],
        'log_return1_sma':['last', min_max_range],
        'log_return1_sms':['last', min_max_range],
        'log_return1_sma_diff':['last', min_max_range],
        'log_return1_sms_diff':['last', min_max_range],
    }
    # These are the features that will be replicated for last 500, 400, 300, 200, 100 seconds
    create_feature_dict_time = {
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
        'total_volume': ['sum','max'],
        'volume_imbalance': ['sum','max']
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 100, add_suffix = True)

    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    df['amount']=df['price']*df['size']
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, np.max, np.min],
        'order_count':[np.sum,np.max],
        'amount':[np.sum,np.max,np.min],
    }
    # These are the features to be replicated for 500, 400,...100 seconds filter
    create_feature_dict_time = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.sum],
    }
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    
    # Create feautres for Tendency and Energy (?); Total 11 features applied to only full seconds (000-599)
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        # new
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        # vol vars
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    df_lr = pd.DataFrame(lis)
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    # Merge all seconds filter
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id','time_id__100'], axis = 1, inplace = True)
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    stock_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility','trade_log_return_realized_volatility']

    time_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200',
                'log_return1_sma_last','log_return1_sma_diff_last','log_return1_sms_last','log_return1_sms_diff_last']
    
    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[stock_cols].agg(['mean','min','max','std']).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the time id
    df_time_id = df.groupby(['time_id'])[time_cols].agg(['mean','min','max','std']).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, mode='train'):
    # Parrallel for loop
    def for_joblib(stock_id):
        print(f'Processing stock {stock_id}')
        # Train
        if mode=='train':
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        elif mode=='test':
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        return df_tmp
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [5]:
%%time
'''
Create all basic features
'''
# Create basic features for train data
if TRAIN==True:
    train, test = read_train_test()
    train_stock_ids = train['stock_id'].unique()[:N_STOCK]
    train = train.merge(preprocessor(train_stock_ids, mode='train'), on = ['row_id'], how = 'left')
    key_cols = ['stock_id','time_id','row_id','target']
    feat_cols = [col for col in train.columns if col not in key_cols]
    train[feat_cols] = train[feat_cols].fillna(train[feat_cols].mean()).fillna(0)
    train = get_time_stock(train)
else:
    train = pd.read_feather(os.path.join(FE_PATH, 'train.f'))

# Create basic features for test data
if TEST_MODE=='test':
    _ , test = read_train_test()
    test_stock_ids = test['stock_id'].unique()[:N_STOCK]
    test = test.merge(preprocessor(test_stock_ids, mode='test'), on=['row_id'], how='left')
elif TEST_MODE=='syn':
    test, _ = read_train_test()
    syn_test_stock_ids = train['stock_id'].sort_values().unique()[:N_SYN_STOCK]
    test = test.merge(preprocessor(syn_test_stock_ids, mode='train'), on=['row_id'], how='inner') # inner join is needed otherwise will return full 420K rows
    syn_test_target = test[['stock_id','time_id','row_id','target']]
    syn_test_target.to_csv('syn_test_target.csv', index=False)
    test = test.drop('target', axis=1)
key_cols = ['stock_id','time_id','row_id','target']
feat_cols = [col for col in test.columns if col not in key_cols]
test[feat_cols] = test[feat_cols].fillna(test[feat_cols].mean()).fillna(0)
test = get_time_stock(test)

Our training set has 428932 rows


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


CPU times: user 2.12 s, sys: 1.37 s, total: 3.49 s
Wall time: 8.49 s


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s finished


In [6]:
%%time
'''
Adding correlated stocks features
'''
# function to get time series for all stocks for similarity calculation in next step
def gen_knn_stock_data(train_test, n_stock, seconds_sample_interval):
    # define parameters
    stock_list = sorted([int(x.split('=')[1]) for x in os.listdir(f'/kaggle/input/optiver-realized-volatility-prediction/book_{train_test}.parquet')])
    stock_list = stock_list[:n_stock]
    # setup base table as all combinations of time x seconds
    time_id_list = pd.DataFrame({'time_id':sorted(load_single_stock(0, train_test, 'book').time_id.unique().tolist())})
    seconds_in_bucket_list = pd.DataFrame({'seconds_in_bucket':range(600)})
    base = time_id_list.merge(seconds_in_bucket_list, how='cross')
    # loop through stocks to get time series data
    ts = []
    for stock_id in stock_list:
        # load data
        book = load_single_stock(stock_id, train_test, 'book')
        # fill NA
        book = base.merge(book, how='left', on=['time_id','seconds_in_bucket'])
        book = book.ffill().bfill()  
        # create series
        book['wap1'] = (book['bid_price1'] * book['ask_size1'] + book['ask_price1'] * book['bid_size1']) / ( book['bid_size1'] +  book['ask_size1'])
        book['wap1_lr'] = np.log(book['wap1']).diff()
        book['wap1_sms50'] = book.groupby('time_id')['wap1_lr'].apply(lambda s: s.rolling(window=50).std())
        # filtering
        idx = book.index.tolist()[::seconds_sample_interval]
        book = book.iloc[idx,:]
        book = book[book['wap1_sms50'].isnull()==False].reset_index(drop=True)
        # centering
        book['wap1_sms50'] = book['wap1_sms50'] - book['time_id'].map(book.groupby('time_id')['wap1_sms50'].median())
        # merge with master table
        book[stock_id] = book['wap1_sms50']
        book = book[['time_id','seconds_in_bucket',stock_id]].set_index(['time_id','seconds_in_bucket'])
        ts.append(book)
        if stock_list.index(stock_id)%20==0:
            print(f'[{datetime.datetime.now()}] Added {stock_list.index(stock_id)} stocks time series.')
    ts = pd.concat(ts, axis=1).reset_index().drop('seconds_in_bucket', axis=1)
    print(f'[{datetime.datetime.now()}] Total time series size is {int(ts.memory_usage().sum() / 1024**2)} MB')
    return ts

# function to top N correlated stocks per stock per time_id
def gen_corr_stock_mapping(data, n_top, show_distance):
    n_top = min(n_top, len(data.columns.tolist()[1:])-1)
    time_id_list = data.time_id.unique().tolist() 
    mapping = []
    for time_id in time_id_list:
        corr = data[data.time_id==time_id].iloc[:,1:].corr().reset_index().rename(columns={'index':'corr_stock_id'})
        for stock_id in corr.columns.tolist()[1:]:
            df = pd.DataFrame({'nearest_stocks': corr[corr.corr_stock_id!=stock_id].sort_values(by=stock_id, ascending=False)['corr_stock_id'].tolist()[:n_top]})
            if show_distance==True:
                df['nearest_stocks_corr'] = corr[corr.corr_stock_id!=stock_id].sort_values(by=stock_id, ascending=False)[stock_id].tolist()[:n_top]
            df['stock_id'] = stock_id
            df['time_id'] = time_id
            mapping.append(df)
        if time_id_list.index(time_id)%500==0:
            print(f'[{datetime.datetime.now()}] Completed correlation for {time_id_list.index(time_id)} time_id.')
    mapping = pd.concat(mapping, axis=0)
    mapping = mapping[['stock_id','time_id','nearest_stocks'] + [col for col in mapping.columns if col not in ['stock_id','time_id','nearest_stocks']]]
    return mapping

# generate new features based on correlated stocks
def gen_corr_stock_feats(data, mapping, target_cols):
    new_feats = pd.merge(data.rename(columns={'stock_id':'nearest_stocks'})[['nearest_stocks','time_id'] + target_cols],
                        mapping, how='inner', on=['nearest_stocks','time_id']).\
                        groupby(['stock_id','time_id'])[target_cols].\
                        mean().\
                        reset_index()
    new_feats.columns = ['stock_id','time_id'] + [f'{col}_corr' for col in new_feats.columns if col not in ['stock_id','time_id']]
    data = pd.merge(data, new_feats, how='left', on=['stock_id','time_id'])
    return data

# correlated stock features
target_cols = ['log_return1_realized_volatility','log_return1_realized_volatility_300','log_return1_realized_volatility_100',
               'log_return1_sma_last','log_return1_sma_diff_last','log_return1_sms_last','log_return1_sms_diff_last',
               'total_volume_sum','volume_imbalance_sum','trade_size_sum','price_spread_sum']

if TRAIN==True:
    corr_mapping_train = gen_corr_stock_mapping(gen_knn_stock_data(train_test='train', n_stock=N_STOCK, seconds_sample_interval=10), n_top=5, show_distance=False)
    train = gen_corr_stock_feats(data=train, mapping=corr_mapping_train, target_cols=target_cols)

if TEST_MODE=='test':
    corr_mapping_test = gen_corr_stock_mapping(gen_knn_stock_data(train_test='test', n_stock=N_STOCK, seconds_sample_interval=10), n_top=5, show_distance=False)
    test = gen_corr_stock_feats(data=test, mapping=corr_mapping_test, target_cols=target_cols)
elif TEST_MODE=='syn':
    corr_mapping_test = gen_corr_stock_mapping(gen_knn_stock_data(train_test='train', n_stock=N_SYN_STOCK, seconds_sample_interval=10), n_top=5, show_distance=False)
    test = gen_corr_stock_feats(data=test, mapping=corr_mapping_test, target_cols=target_cols)

[2021-09-27 16:17:20.284963] Added 0 stocks time series.
[2021-09-27 16:17:20.288102] Total time series size is 0 MB
[2021-09-27 16:17:20.293612] Completed correlation for 0 time_id.
CPU times: user 55.1 ms, sys: 4.17 ms, total: 59.2 ms
Wall time: 61.8 ms


In [7]:
%%time
'''
Adding features for Time Series shape clustering
'''
!pip install ../input/tslearn052/tslearn-0.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl
from tslearn.clustering import TimeSeriesKMeans, KShape
from tslearn.generators import random_walks
from tslearn.utils import to_time_series_dataset
from tslearn.barycenters import dtw_barycenter_averaging
import math
import random

def preprocess_single_stock(stock_id, train_test, n_time_id_sample, seconds_sample_interval, remove_first_n_seconds, delete_unused_cols):
    # load data
    book = load_single_stock(stock_id, train_test, 'book')
    # ffill and bfill
    time_id_list =  pd.DataFrame({'time_id':book.time_id.unique().tolist()})
    seconds_in_bucket_list = pd.DataFrame({'seconds_in_bucket':range(600)})
    base = time_id_list.merge(seconds_in_bucket_list, how='cross')
    book = base.merge(book, how='left', on=['time_id','seconds_in_bucket'])
    book = book.ffill().bfill()
    # sampling time_id
    if n_time_id_sample >= 0:
        time_id_samples = random.sample(book['time_id'].unique().tolist(), n_time_id_sample)
        book = book[book.time_id.isin(time_id_samples)]
    # create time series
    book['wap1'] = (book['bid_price1'] * book['ask_size1'] + book['ask_price1'] * book['bid_size1']) / ( book['bid_size1'] +  book['ask_size1'])
    book['wap1_sma50'] = book.groupby('time_id')['wap1'].apply(lambda s: s.rolling(window=50).mean())
    book['wap1_lr'] = np.log(book['wap1']).diff()
    book['wap1_sms50'] = book.groupby('time_id')['wap1_lr'].apply(lambda s: s.rolling(window=50).std())
    book['total_volume'] = book['ask_size1'] + book['ask_size2'] + book['bid_size1'] + book['bid_size2']
    book['total_volume'] = (book['total_volume'] - book['total_volume'].mean()) / book['total_volume'].std()
    book['total_volume_sma60'] = book.groupby('time_id')['total_volume'].apply(lambda s: s.rolling(window=60).mean())
    book['volume_imbalance'] = (book['ask_size1'] + book['ask_size2']) - (book['bid_size1'] + book['bid_size2'])
    book['volume_imbalance'] = (book['volume_imbalance'] - book['volume_imbalance'].mean()) / book['volume_imbalance'].std()
    book['volume_imbalance_sma80'] = book.groupby('time_id')['volume_imbalance'].apply(lambda s: s.rolling(window=80).mean())
    # remove first few entries to avoid NA
    book = book[book.seconds_in_bucket >= remove_first_n_seconds].reset_index(drop=True)
    # seconds interval filtering
    idx = book.index.tolist()[::seconds_sample_interval]
    book = book.iloc[idx,:]
    # select only relevant columns
    book['stock_id'] = stock_id
    if delete_unused_cols==True:
        book = book[['stock_id','time_id','seconds_in_bucket','wap1_sma50','wap1_sms50','total_volume_sma60','volume_imbalance_sma80']]
    return book

def tagging_cluster_all_stocks(n_stock, train_test, n_time_id_sample, seconds_sample_interval, remove_first_n_seconds, delete_unused_cols):
    stock_list = sorted([int(x.split('=')[1]) for x in os.listdir(f'/kaggle/input/optiver-realized-volatility-prediction/book_{train_test}.parquet')])[:n_stock]
    all_stocks_tagging = []
    for stock_id in stock_list:
        ts = preprocess_single_stock(stock_id=stock_id, train_test=train_test, n_time_id_sample=n_time_id_sample, 
                                     seconds_sample_interval=seconds_sample_interval, remove_first_n_seconds=remove_first_n_seconds, 
                                     delete_unused_cols=delete_unused_cols)
        tagging = ts[['stock_id','time_id']].drop_duplicates()
        path = '/kaggle/input/volatility-ts-clustering-models-v1' # define KMeans model path and names
        clust_models = ['clust_wap1_sma50','clust_wap1_sms50','clust_total_volume_sma60','clust_volume_imbalance_sma80']
        for name in clust_models[:4]: # loop through models to perform tagging
            model = TimeSeriesKMeans.from_pickle(os.path.join(path, f'{name}.p'))
            tagging[name] = model.predict(dataframe_to_ts(data=ts, series=name.replace('clust_','')))
            tagging[name] = tagging[name].astype(int)
        all_stocks_tagging.append(tagging)
        print(f'[{datetime.datetime.now()}] Completed {train_test} data tagging for stock {stock_id}')
    all_stocks_tagging = pd.concat(all_stocks_tagging, axis=0).reset_index(drop=True)
    return all_stocks_tagging

def dataframe_to_ts(data, series):
    ts = [data[(data.stock_id==stock_id) & (data.time_id==time_id) & (data[series].isnull()==False)][series].tolist() for stock_id, time_id in data[['stock_id','time_id']].drop_duplicates().to_records(index=False).tolist()]
    ts = to_time_series_dataset(ts)
    return ts

# create the new features
# tagging_train = gen_ts_clustering_feats(train_test='train', n_stock=N_STOCK)
if TRAIN==True:
    tagging_train = pd.read_csv('/kaggle/input/stock-cluster-tagging-train/tagging_train.csv') # pre-computed for training set
    train = pd.merge(train, tagging_train, how='left', on=['stock_id','time_id'])
    cols = [col for col in train.columns if col[:6]=='clust_']
    train[cols] = train[cols].fillna(0)

if TEST_MODE=='test':
    tagging_test = tagging_cluster_all_stocks(n_stock=N_STOCK, train_test='test', n_time_id_sample=-1, seconds_sample_interval=10, remove_first_n_seconds=10, delete_unused_cols=True)
    test = pd.merge(test, tagging_test, how='left', on=['stock_id','time_id'])
elif TEST_MODE=='syn':
    tagging_test = tagging_cluster_all_stocks(n_stock=N_SYN_STOCK, train_test='train', n_time_id_sample=-1, seconds_sample_interval=10, remove_first_n_seconds=10, delete_unused_cols=True)
    test = pd.merge(test, tagging_test, how='left', on=['stock_id','time_id'])
cols = [col for col in test.columns if col[:6]=='clust_']
test[cols] = test[cols].fillna(0)

Processing /kaggle/input/tslearn052/tslearn-0.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl


/opt/conda/lib/python3.7/site-packages/tslearn/clustering/kmeans.py:17: UserWarning: Scikit-learn <0.24 will be deprecated in a future release of tslearn
  "Scikit-learn <0.24 will be deprecated in a "


[2021-09-27 16:17:49.794238] Completed test data tagging for stock 0
CPU times: user 1.59 s, sys: 254 ms, total: 1.84 s
Wall time: 29.4 s


In [8]:
'''
Add Stock Clustering as categorical feature
'''
from sklearn.cluster import AgglomerativeClustering
# prepare data for clustering
data = pd.pivot_table(train, values='target', index='time_id', columns='stock_id', aggfunc=np.sum)
data = data.fillna(data.mean()).T
# model fitting
model = AgglomerativeClustering(n_clusters=6, linkage='ward')
label = model.fit_predict(data)
print(pd.Series(label).value_counts())
# create stock-label mapping table
data['stock_clustering_label'] = label
data = data.reset_index().rename(columns={'index':'stock_id'})
data = data[['stock_id','stock_clustering_label']]
# add label to train / test
if TRAIN==True:
    train = pd.merge(train, data, how='left', on='stock_id')
test = pd.merge(test, data, how='left', on='stock_id')

3    33
2    30
5    27
1    13
0     8
4     1
dtype: int64


In [9]:
'''
Add Time Clustering as categorical feature
'''

from sklearn.cluster import KMeans
feature_list = ['wap1_sum','log_return1_realized_volatility','log_return1_realized_volatility_200','log_return1_realized_volatility_400','total_volume_sum','volume_imbalance_sum','trade_size_sum','trade_order_count_sum','trade_log_return_realized_volatility']
log_feats = ['log_return1_realized_volatility','log_return1_realized_volatility_200','log_return1_realized_volatility_400','total_volume_sum','trade_size_sum','trade_order_count_sum','trade_log_return_realized_volatility']

if TRAIN==True:
    # prepare data for clustering
    train_time_feature = train.groupby('time_id')[feature_list].mean().reset_index()
    train_time_feature[log_feats] = np.log(train_time_feature[log_feats])
    train_time_feature = fill_inf_nan(train_time_feature)
    # kmeans clustering
    kmeans = KMeans(n_clusters=8, random_state=SEED).fit(train_time_feature[feature_list])
    # transformation for train
    train_time_feature['time_clustering_label'] = kmeans.predict(train_time_feature[feature_list])
    train_time_feature = train_time_feature[['time_id','time_clustering_label']]
    train = pd.merge(train, train_time_feature, how='left', on='time_id')
    # transformation for test
    test_time_feature = test.groupby('time_id')[feature_list].mean().reset_index()
    test_time_feature[log_feats] = np.log(test_time_feature[log_feats])
    test_time_feature = fill_inf_nan(test_time_feature)
    test_time_feature['time_clustering_label'] = kmeans.predict(test_time_feature[feature_list])
    test_time_feature = test_time_feature[['time_id','time_clustering_label']]
    test = pd.merge(test, test_time_feature, how='left', on='time_id')
    # save model
    pickle.dump(kmeans, open(f'time_clustering_kmeans_model.p', 'wb'))
else:
    # transformation for test
    test_time_feature = test.groupby('time_id')[feature_list].mean().reset_index()
    test_time_feature[log_feats] = np.log(test_time_feature[log_feats])
    test_time_feature = fill_inf_nan(test_time_feature)
    kmeans = pickle.load(open(os.path.join(FE_PATH, f'time_clustering_kmeans_model.p'), 'rb'))
    test_time_feature['time_clustering_label'] = kmeans.predict(test_time_feature[feature_list])
    test_time_feature = test_time_feature[['time_id','time_clustering_label']]
    test = pd.merge(test, test_time_feature, how='left', on='time_id')

In [10]:
%%time
'''
Time-Series LSTM AutoEncoding features
'''
from tensorflow.keras import metrics
from tensorflow import keras
import tensorflow as tf
from numpy.random import seed
from tensorflow.keras.models import Model

# function to load a book/trade train/test single stock file
def load_single_stock(stock_id, train_test, book_trade):
    path = f'/kaggle/input/optiver-realized-volatility-prediction/{book_trade}_{train_test}.parquet/stock_id={str(stock_id)}'
    filename = os.path.join(path, os.listdir(path)[0])
    return pd.read_parquet(filename)

def preprocess_single_stock(stock_id, train_test, n_time_id_sample, seconds_sample_interval):
    # load data
    book = load_single_stock(stock_id, train_test, 'book')
    trade = load_single_stock(stock_id, train_test, 'trade')
    # ffill and bfill
    time_id_list =  pd.DataFrame({'time_id':book.time_id.unique().tolist()})
    seconds_in_bucket_list = pd.DataFrame({'seconds_in_bucket':range(600)})
    base = time_id_list.merge(seconds_in_bucket_list, how='cross')
    book = base.merge(book, how='left', on=['time_id','seconds_in_bucket'])
    book = book.ffill().bfill()
    trade = base.merge(trade, how='left', on=['time_id','seconds_in_bucket'])
    trade[['price']] = trade[['price']].ffill().bfill()
    trade[['size','order_count']] = trade[['size','order_count']].fillna(0)
    # joining book and trade
    df = pd.merge(book, trade, how='inner', on=['time_id','seconds_in_bucket'])
    # sampling time_id
    if n_time_id_sample >= 0:
        time_id_samples = random.sample(df['time_id'].unique().tolist(), n_time_id_sample)
        df = df[df.time_id.isin(time_id_samples)]
    # smoothing all time series
    ts = [c for c in df if c not in ['time_id','seconds_in_bucket']]
    for c in ts:
        df[c] = df.groupby('time_id')[c].apply(lambda s: s.rolling(window=50).mean())
    # seconds interval filtering
    idx = df.index.tolist()[::seconds_sample_interval]
    df = df.iloc[idx,:]
    # remove NA entries due to moving average
    df = df[df.isnull().sum(axis=1)==0].reset_index(drop=True)
    # select only relevant columns
    df['stock_id'] = stock_id
    df = df[['stock_id'] + [c for c in df if c not in ['stock_id']]]
    df = df.drop('seconds_in_bucket', axis=1)
    return df

def preprocess_all_stocks(n_stock, train_test, n_time_id_sample, seconds_sample_interval, reduce_mem):
    stock_list = sorted([int(x.split('=')[1]) for x in os.listdir(f'/kaggle/input/optiver-realized-volatility-prediction/book_{train_test}.parquet')])
    stock_list = stock_list[:n_stock]
    df = []
    for stock_id in stock_list:
        df.append(preprocess_single_stock(stock_id, train_test, n_time_id_sample, seconds_sample_interval))
        if stock_list.index(stock_id) % 10 == 0:
            print(f'[{datetime.datetime.now()}] Processed {stock_list.index(stock_id) % 10} stock_ids')
    df = pd.concat(df, axis=0).reset_index(drop=True)
    print(f'[{datetime.datetime.now()}] Completed data preprocessing')
    
    # scaling
    num_feats = [c for c in df if c not in ['stock_id','time_id']]
    scalers = pickle.load(open(os.path.join('/kaggle/input/volatility-time-series-autoencoder-data', 'ts_ae_stdscalers.p'), 'rb'))
    for c in num_feats:
        df[[c]] = scalers[c].transform(df[[c]])
        print(f'[{datetime.datetime.now()}] Scaled {c}')
    print(f'[{datetime.datetime.now()}] Completed data scaling')
    
    # memory reduction
    if reduce_mem==True:
        df = reduce_mem_usage(df)
        print(f'[{datetime.datetime.now()}] Completed memory reduction')
    return df

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:5] == 'float':
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

# create dataset
if TEST_MODE=='test':
    data_test = preprocess_all_stocks(n_stock=N_STOCK, train_test='test', n_time_id_sample=-1, seconds_sample_interval=10, reduce_mem=True)
elif TEST_MODE=='syn':
    data_test = preprocess_all_stocks(n_stock=N_SYN_STOCK, train_test='train', n_time_id_sample=-1, seconds_sample_interval=10, reduce_mem=True)
print(f'[{datetime.datetime.now()}] Completed data preprocessing')

# remove price
# data_train = data_train.drop(['ask_price1','bid_price2','ask_price2'], axis=1)
data_test = data_test.drop(['ask_price1','bid_price2','ask_price2'], axis=1)

# convert to np array
data_test_keys = data_test[['stock_id','time_id']].drop_duplicates()
n_split = data_test_keys.shape[0]
data_test = np.array(np.split(data_test.iloc[:,2:].values, n_split))
print(f'Shape of data_test is {data_test.shape}')

# load pre-trained encoder
path = os.path.join('/kaggle/input/volatility-ts-encoding-version-5', 'TimeSeries_Encoder')
ts_encoder = tf.keras.models.load_model(path)
encoding_dim = 32

# encoding
if TRAIN==True:
    data_train_keys = pd.read_csv('/kaggle/input/volatility-ts-encoding-version-5/train_encoded.csv')
    print(f'Shape of train encoding is {data_train_keys.shape}')
    train = pd.merge(train, data_train_keys, how='left', on=['stock_id','time_id'])
    del data_train_keys
    gc.collect()
data_test_keys[[f'ts_ae{i}' for i in range(encoding_dim)]] = ts_encoder.predict(data_test)
print(f'Shape of test encoding is {data_test_keys.shape}')
test = pd.merge(test, data_test_keys, how='left', on=['stock_id','time_id'])
del data_test, data_test_keys
gc.collect()

[2021-09-27 16:17:54.877264] Processed 0 stock_ids
[2021-09-27 16:17:54.878171] Completed data preprocessing
[2021-09-27 16:17:54.885460] Scaled bid_price1
[2021-09-27 16:17:54.888540] Scaled ask_price1
[2021-09-27 16:17:54.891484] Scaled bid_price2
[2021-09-27 16:17:54.894323] Scaled ask_price2
[2021-09-27 16:17:54.897167] Scaled bid_size1
[2021-09-27 16:17:54.900038] Scaled ask_size1
[2021-09-27 16:17:54.903345] Scaled bid_size2
[2021-09-27 16:17:54.906330] Scaled ask_size2
[2021-09-27 16:17:54.909233] Scaled price
[2021-09-27 16:17:54.912243] Scaled size
[2021-09-27 16:17:54.915083] Scaled order_count
[2021-09-27 16:17:54.915135] Completed data scaling
Memory usage after optimization is: 0.00 MB
Decreased by 41.4%
[2021-09-27 16:17:54.923469] Completed memory reduction
[2021-09-27 16:17:54.923539] Completed data preprocessing
Shape of data_test is (1, 55, 8)
Shape of test encoding is (1, 34)
CPU times: user 9.48 s, sys: 1.74 s, total: 11.2 s
Wall time: 18.1 s


11501

In [11]:
# Create "tau" features
if TRAIN==True:
    train['size_tau'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique'] )
    train['size_tau_400'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_400'] )
    train['size_tau_300'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_300'] )
    train['size_tau_200'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_200'] )
    train['size_tau2'] = np.sqrt( 1/ train['trade_order_count_sum'] )
    train['size_tau2_400'] = np.sqrt( 0.33/ train['trade_order_count_sum'] )
    train['size_tau2_300'] = np.sqrt( 0.5/ train['trade_order_count_sum'] )
    train['size_tau2_200'] = np.sqrt( 0.66/ train['trade_order_count_sum'] )
    train['size_tau2_d'] = train['size_tau2_400'] - train['size_tau2']
    cols = [col for col in train.columns if 'tau' in col]
    train[cols] = train[cols].replace([np.inf, -np.inf], np.nan)
    train[cols] = train[cols].fillna(train[cols].mean())

test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )
test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']
cols = [col for col in test.columns if 'tau' in col]
test[cols] = test[cols].replace([np.inf, -np.inf], np.nan)
test[cols] = test[cols].fillna(test[cols].mean())

In [12]:
'''
Stock cluster features
For each time_id, statistics for each of 5 clusters (for only that time_id) will be added
'''
from sklearn.cluster import KMeans
from collections import Counter

# clustering based on target correlation matrix
n_clust = 7
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')
corr = train_p.corr()
kmeans = KMeans(n_clusters=n_clust, random_state=SEED).fit(corr.values)
print(Counter(kmeans.labels_))

# output the list of stocks for each cluster
stock_groups = []
ids = corr.index
for n in range(n_clust):
    stock_groups.append([(x-1) for x in ((ids+1)*(kmeans.labels_==n)) if x > 0])

def gen_clust_feats(stock_groups, train, test):
    df_train = []
    df_test = []
    for i in range(len(stock_groups)):
        group = stock_groups[i]
        newDf = train.loc[train['stock_id'].isin(group)]
        newDf = newDf.groupby(['time_id']).agg(np.nanmean)
        newDf.loc[:,'stock_id'] = f'c{i}'
        df_train.append(newDf)

        newDf = test.loc[test['stock_id'].isin(group)]    
        newDf = newDf.groupby(['time_id']).agg(np.nanmean)
        newDf.loc[:,'stock_id'] = f'c{i}'
        df_test.append(newDf)

    df_train = pd.concat(df_train).reset_index()
    df_train = df_train[[c for c in df_train if c!='target']]
    df_test = pd.concat(df_test).reset_index()

    df_test = pd.concat([df_test, df_train[(~df_train.stock_id.isin(df_test.stock_id)) & (df_train.time_id==5)]]) # suspicious; probably this is to ensure that test data contains all stocks; it works on the assumption that time_id 5 is not required for submission
    df_train = df_train.pivot(index='time_id', columns='stock_id')
    df_train.columns = ["_".join(x) for x in df_train.columns.ravel()]
    df_train.reset_index(inplace=True)

    df_test = df_test.pivot(index='time_id', columns='stock_id')
    df_test.columns = ["_".join(x) for x in df_test.columns.ravel()]
    df_test.reset_index(inplace=True)

    # add to base features
    target_cols = ['time_id'] + [f'{col}_c{i}' for i in [0,2,3,4,6] for col in ['log_return1_realized_volatility','total_volume_sum','trade_size_sum','trade_order_count_sum',
                                                                                'price_spread_sum','bid_spread_sum','ask_spread_sum','volume_imbalance_sum','bid_ask_spread_sum','size_tau2']]
    train = pd.merge(train, df_train[target_cols], how='left', on='time_id')
    test = pd.merge(test, df_test[target_cols], how='left', on='time_id')

    # free memory
    import gc
    del df_train, df_test
    gc.collect()
    return train, test

if TRAIN==True:
    train, test = gen_clust_feats(stock_groups=stock_groups, train=train[[c for c in train if c[-3:-1]!='_c']], test=test)
else:
    _ , test = gen_clust_feats(stock_groups=stock_groups, train=train, test=test)
    
train = train.fillna(train.mean()).fillna(0)
test = test.fillna(test.mean()).fillna(0)

Counter({3: 32, 0: 31, 4: 19, 2: 19, 6: 8, 1: 2, 5: 1})


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:47: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.


In [13]:
# # remove highly correlated features
# corr = train[[col for col in train.columns if col[-3:-1]!='_c' and col[:6]!='clust_' and col not in key_cols]].corr()
# all_features = corr.columns.tolist()
# print(f'Original feats is {len(all_features)}')
# correlated_features = set()
# for i in range(len(corr.columns)):
#     for j in range(i):
#         if corr.iloc[i, j] >= 0.99:
#             correlated_features.add(corr.columns[i])
# print(f'Removed feats is {len(correlated_features)}')
# train = train.drop(correlated_features, axis=1)
# test = test.drop(correlated_features, axis=1)

In [14]:
# final fillna
train = fill_inf_nan(train)
test = fill_inf_nan(test)

In [15]:
# Print number of features
colNames = [col for col in list(train.columns) if col not in ["stock_id", "time_id", "target", "row_id"]]
print(f'Number of features is {len(colNames)}')

Number of features is 301


### NN Dataset

In [16]:
# prepare NN dataset

# features that requires quantile transformation for NN model
nn_qt_feats = [col for col in train.columns if \
               col[-3:-1]!='_c' and \
               col[:6]!='clust_' and \
               'clustering_label' not in col and \
               col not in ['stock_id','time_id','row_id','target']]

if TRAIN==True:
    train_nn=train[nn_qt_feats].copy()
else:
    train_nn = pd.read_feather(os.path.join(FE_PATH, 'train_nn.f'))
test_nn=test[nn_qt_feats].copy()

# quantile transformation
if TRAIN==True:
    qt = QuantileTransformer(random_state=SEED, n_quantiles=2000, output_distribution='normal')
    qt.fit(train_nn[nn_qt_feats])
    train_nn[nn_qt_feats] = qt.transform(train_nn[nn_qt_feats])
    test_nn[nn_qt_feats] = qt.transform(test_nn[nn_qt_feats])
    pickle.dump(qt, open(f'quantile_transformer.p', 'wb'))
else:
    qt = pickle.load(open(os.path.join(FE_PATH, 'quantile_transformer.p'), 'rb'))
    test_nn[nn_qt_feats] = qt.transform(test_nn[nn_qt_feats])

# reset key columns
train_nn[['stock_id','time_id','row_id','target']] = train[['stock_id','time_id','row_id','target']].reset_index(drop=True)
test_nn[['stock_id','time_id','row_id']] = test[['stock_id','time_id','row_id']].reset_index(drop=True)

# add back stock and time clustering labels
stock_time_clust_label_feats = [c for c in train if 'clustering_label' in c]
train_nn[stock_time_clust_label_feats] = train[stock_time_clust_label_feats]
test_nn[stock_time_clust_label_feats] = test[stock_time_clust_label_feats]

# generate stock cluster features
if TRAIN==True:
    train_nn, test_nn = gen_clust_feats(stock_groups=stock_groups, train=train_nn, test=test_nn)
else:
    _ , test_nn = gen_clust_feats(stock_groups=stock_groups, train=train_nn, test=test_nn)

# remove correlated features
# correlated_features = [c for c in correlated_features if c in train_nn.columns.tolist()]
# train_nn = train_nn.drop(correlated_features, axis=1)
# test_nn = test_nn.drop(correlated_features, axis=1)

# fillna with mean
train_nn = fill_inf_nan(train_nn)
test_nn = fill_inf_nan(test_nn)

# stock_id label encoding (required for NN embedding)
nn_cat_cols = ['stock_id'] + stock_time_clust_label_feats
for col in nn_cat_cols:
    if TRAIN==True:
        encoder = LabelEncoder()
        encoder.fit(train_nn[col].values)
        train_nn[col] = encoder.transform(train_nn[col].values)
        test_nn[col] = encoder.transform(test_nn[col].values)
        pickle.dump(encoder, open(f'nn_label_encoder_{col}.p', 'wb'))
    else:
        encoder = pickle.load(open(os.path.join(FE_PATH, f'nn_label_encoder_{col}.p'), 'rb'))
        test_nn[col] = encoder.transform(test_nn[col].values)
        
# min max scaling
nn_num_feats = [c for c in train_nn if c not in ['stock_id','time_id','row_id','target'] and 'clustering_label' not in c]
if TRAIN==True:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(train_nn[nn_num_feats])
    train_nn[nn_num_feats] = scaler.transform(train_nn[nn_num_feats])
    pickle.dump(scaler, open(f'nn_min_max_scaler.p', 'wb'))
    test_nn[nn_num_feats] = scaler.transform(test_nn[nn_num_feats])
else:
    scaler = pickle.load(open(os.path.join(FE_PATH, 'nn_min_max_scaler.p'), 'rb'))
    test_nn[nn_num_feats] = scaler.transform(test_nn[nn_num_feats])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:47: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.


### TabNet Dataset

In [17]:
# define dataset for TabNet
train_tbn = train_nn.copy()
test_tbn = test_nn.copy()

# identify categorical and numerical columns
cat_cols = ['stock_id'] + stock_time_clust_label_feats
num_cols = [c for c in train_tbn if c not in ['stock_id','time_id','row_id','target'] and c not in stock_time_clust_label_feats]

# label encoding catergorical cols
for col in cat_cols:
    if TRAIN==True:
        encoder = LabelEncoder()
        encoder.fit(train_tbn[col].values)
        train_tbn[col] = encoder.transform(train_tbn[col].values)
        test_tbn[col] = encoder.transform(test_tbn[col].values)
        pickle.dump(encoder, open(f'tabnet_label_encoder_{col}.p', 'wb'))
    else:
        encoder = pickle.load(open(os.path.join(FE_PATH, f'tabnet_label_encoder_{col}.p'), 'rb'))
        test_tbn[col] = encoder.transform(test_tbn[col].values)
    
# standard scaling numerical cols
if TRAIN==True:
    scaler = StandardScaler()
    scaler.fit(train_tbn[num_cols])
    train_tbn[num_cols] = scaler.transform(train_tbn[num_cols])
    test_tbn[num_cols] = scaler.transform(test_tbn[num_cols])
    pickle.dump(scaler, open(f'tabnet_std_scaler.p', 'wb'))
else:
    scaler = pickle.load(open(os.path.join(FE_PATH, 'tabnet_std_scaler.p'), 'rb'))
    test_tbn[num_cols] = scaler.transform(test_tbn[num_cols])

In [18]:
# export
if TRAIN==True:
    train.reset_index(drop=True).to_feather('train.f')
    test.reset_index(drop=True).to_feather('test.f')
    train_nn.reset_index(drop=True).to_feather('train_nn.f')
    test_nn.reset_index(drop=True).to_feather('test_nn.f')
    train_tbn.reset_index(drop=True).to_feather('train_tbn.f')
    test_tbn.reset_index(drop=True).to_feather('test_tbn.f')

In [19]:
print(train.shape)
print(test.shape)
print(train_nn.shape)
print(test_nn.shape)
print(train_tbn.shape)
print(test_tbn.shape)

(428932, 305)
(3, 304)
(428932, 301)
(3, 300)
(428932, 301)
(3, 300)


### Change Log
* Version 40: LGBM: stock clust + time clust; NN: stock clust; TabNet: time clust
* Version 41: LGBM: time clust; NN: stock clust; TabNet: time clust
* Version 42: Clipping min=0
* Version 44: Ensembling - Random Forest
* Version 45: Ensembling - ElasticNet with correlated stocks
* Version 46: FE with enhanced error handling
* Version 47: Ensembling - ElasticNet with correlated stocks (not forcing positive weights)
* Version 48: Ensembling - Random Forest with correlated stocks
* Version 49: Ensembling - ElasticNet with correlated stocks (not forcing positive weights)
* Version 50: Testing of new FE dataset with syn testing (only run LGBM)
* Version 51: Removing >0.99 correlated features
* Version 53:
    - Optimized LGBM + TabNet params
    - Remove correlated feats for TabNet
    - Ensembling - Random Forest with correlated stocks
* Version 54: ElasticNet

Feature:
* Stock / Time clustering - direclty put clustering centroid as feature
* Correlation between time series as a feature
* Remove highly correlated features
* Quarticity https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/267096

Model:
* RF meta model
* Add correlated stock prediction to meta model
* Final HP Tuning of all models
* AutoEncoder+1dCNN from public
* 1D CNN
* 2D CNN https://towardsdatascience.com/how-to-encode-time-series-into-images-for-financial-forecasting-using-convolutional-neural-networks-5683eb5c53d9
    - GAF for time series to image transformation
    - 11 features corresponding to 11 channels
* LSTM + Dense layer regression

Ensembling:

In [20]:
INFERENCE = True
TEST_MODE = 'test'

FE_PATH = '/kaggle/input/volatility-fe-output-version-15'
BASE_MODEL_PATH = '/kaggle/input/volatility-model-training-output-version-54' # this is for fixing the pre-trained base models (for submission only)

SEED = 1111
N_FOLD = 5

LGBM_NUM_BOOST = 3000
NN_EPOCH = 1000
TABNET_EPOCH = 1000
# LGBM_NUM_BOOST = 1
# NN_EPOCH = 1
# TABNET_EPOCH = 1

In [21]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np
import glob
import os
import gc
import datetime
import pickle

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib
import random
pd.set_option('display.max_columns', None)

# set seed
def seed_everything(seed=SEED):
    import torch
    import random
    import os
    import numpy as np
    import tensorflow as tf
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    tf.random.set_seed(SEED)
seed_everything()

In [22]:
# df = pd.DataFrame({'a':[1,2,3,4,5], 'b':['a','a','b','b','c']})
# df.groupby('b')['a'].agg(lambda s:s.max()-s.min())

In [23]:
# import from FE script
if INFERENCE==False:
    train = pd.read_feather(os.path.join(FE_PATH, 'train.f'))
    test = pd.read_feather(os.path.join(FE_PATH, 'test.f'))
    print(f'Train data shape is {train.shape}')
    print(f'Test data shape is {test.shape}')

In [24]:
# define the keys of each dataset which are preserved in the whole notebook
train_key_cols = ['stock_id','time_id','row_id','target']
test_key_cols = ['stock_id','time_id','row_id']
train_keys = train[train_key_cols].reset_index(drop=True)
test_keys = test[test_key_cols].reset_index(drop=True)
y = train['target']

In [25]:
print(train.shape)
print(y.shape)

(428932, 305)
(428932,)


# LGBM Model

In [26]:
%%time
from sklearn.model_selection import KFold, GroupKFold
import lightgbm as lgb

lgb_features = [col for col in train.columns if col not in ["time_id", "target", "row_id"] and 'ts_ae' not in col]

params0 = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': 60,
    'max_bin': 250,
    'min_data_in_leaf':422,
    'learning_rate': 0.014565641020775516,
    'subsample': 0.6563801192981948,
    'subsample_freq': 1,
    'feature_fraction': 0.525513036358404,
    'lambda_l1': 8.177995270216595,
    'lambda_l2': 3.8822889556906657,
    'categorical_column': [lgb_features.index('stock_id'),
                           lgb_features.index('clust_wap1_sma50'),
                           lgb_features.index('clust_wap1_sms50'),
                           lgb_features.index('clust_total_volume_sma60'),
                           lgb_features.index('clust_volume_imbalance_sma80'),
                           lgb_features.index('stock_clustering_label'),
                           lgb_features.index('time_clustering_label')],
    'seed': SEED,
    'feature_fraction_seed': SEED,
    'bagging_seed': SEED,
    'drop_seed': SEED,
    'data_random_seed': SEED,
    'n_jobs':-1,
    'verbose': -1}

# Function to early stop with root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

def train_and_evaluate_lgb(train, test, params):  
    print(f'Number of features considered for LightGBM is {len(lgb_features)}')
    oof_pred = np.zeros(train.shape[0])
    test_pred = []
    kfold = GroupKFold(n_splits=N_FOLD)
    for fold, (trn_idx, val_idx) in enumerate(kfold.split(train, y, train_keys.time_id)):
        
        if INFERENCE==False:
            print(f'Training fold {fold}')
            x_train, x_val = train.iloc[trn_idx], train.iloc[val_idx]
            y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]
            train_weights = 1 / np.square(y_train)
            val_weights = 1 / np.square(y_val)
            train_dataset = lgb.Dataset(x_train[lgb_features], y_train, weight=train_weights)
            val_dataset = lgb.Dataset(x_val[lgb_features], y_val, weight=val_weights)
            model = lgb.train(params=params,
                              num_boost_round=LGBM_NUM_BOOST,
                              train_set=train_dataset, 
                              valid_sets=[train_dataset, val_dataset], 
                              verbose_eval=250,
                              early_stopping_rounds=50,
                              feval=feval_rmspe)
            # predictions
            oof_pred[val_idx] = model.predict(x_val[lgb_features])
            test_pred.append(model.predict(test[lgb_features]))
            # save model
            pickle.dump(model, open(f'lgbm_fold{fold}.p', 'wb'))
        
        elif INFERENCE==True:
            print(f'Inferring fold {fold}')
            model = pickle.load(open(os.path.join(BASE_MODEL_PATH, f'lgbm_fold{fold}.p'), 'rb'))
            test_pred.append(model.predict(test[lgb_features]))
    # Return test predictions
    return oof_pred, test_pred

# Traing and evaluate
oof_pred_lgb, test_pred_lgb = train_and_evaluate_lgb(train, test, params0)
cv_score_lgb = round(rmspe(train['target'], oof_pred_lgb), 5)
print(f'LGBM averaged CV score is {cv_score_lgb}')

del train
gc.collect()

Number of features considered for LightGBM is 270
Inferring fold 0
Inferring fold 1
Inferring fold 2
Inferring fold 3
Inferring fold 4
LGBM averaged CV score is 1.0
CPU times: user 1.44 s, sys: 120 ms, total: 1.56 s
Wall time: 2.86 s


77

# NN Model

In [27]:
if INFERENCE==False:
    train_nn = pd.read_feather(os.path.join(FE_PATH, 'train_nn.f'))
    test_nn = pd.read_feather(os.path.join(FE_PATH, 'test_nn.f'))

In [28]:
from numpy.random import seed
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras import backend as K
seed(SEED)
tf.random.set_seed(SEED)

def root_mean_squared_per_error(y_true, y_pred):
         return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=7, verbose=0,
    mode='min')

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [29]:
# kfold based on the knn++ algorithm
out_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
out_train = out_train[out_train.time_id.isin(train_nn.time_id.unique())]
out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')
out_train = out_train.fillna(out_train.mean())
# code to add the just the read data after first execution
# data separation based on knn ++
index = []
totDist = []
values = []
# generates a matriz with the values of 
mat = out_train.values
scaler = MinMaxScaler(feature_range=(-1, 1))
mat = scaler.fit_transform(mat)
nind = int(mat.shape[0] / N_FOLD) # number of individuals
# adds index in the last column
mat = np.c_[mat,np.arange(mat.shape[0])]
lineNumber = np.random.choice(np.array(mat.shape[0]), size=N_FOLD, replace=False)
lineNumber = np.sort(lineNumber)[::-1]
for n in range(N_FOLD):
    totDist.append(np.zeros(mat.shape[0]-N_FOLD))
# saves index
for n in range(N_FOLD):
    values.append([lineNumber[n]])    
s=[]
for n in range(N_FOLD):
    s.append(mat[lineNumber[n],:])
    mat = np.delete(mat, obj=lineNumber[n], axis=0)
for n in range(nind-1):    
    luck = np.random.uniform(0,1,N_FOLD)
    for cycle in range(N_FOLD):
         # saves the values of index           
        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)
        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
        totDist[cycle] += sumDist        
        # probabilities
        f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totdist
        j = 0
        kn = 0
        for val in f:
            j += val        
            if (j > luck[cycle]): # the column was selected
                break
            kn +=1
        lineNumber[cycle] = kn
        # delete line of the value added    
        for n_iter in range(N_FOLD):
            totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
            j= 0
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis=0)
for n_mod in range(N_FOLD):
    values[n_mod] = out_train.index[values[n_mod]]

In [30]:
# NN Model Architecture

#https://bignerdranch.com/blog/implementing-swish-activation-function-in-keras/
from keras.backend import sigmoid
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation

def swish(x, beta = 1):
    return (x * sigmoid(beta * x))
get_custom_objects().update({'swish': Activation(swish)})

def create_nn_model(hidden_units):
    # initialize input list (later to be concatenated)
    raw_input_list = []
    concat_input_list = []
    
    # Each instance will consist of two inputs: a single user id, and a single movie id
    num_input = keras.Input(shape=(len(numerical_feats),), name='num_data')
    raw_input_list.append(num_input)
    concat_input_list.append(num_input)

    # stock_id embedding
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    stock_id_embedded = keras.layers.Embedding(stock_id_orig_dim, stock_id_emb_dim, input_length=1, name='stock_id_embedding')(stock_id_input)
    stock_id_flattened = keras.layers.Flatten()(stock_id_embedded)
    raw_input_list.append(stock_id_input)
    concat_input_list.append(stock_id_flattened)
    
    # stock clustering embedding
    stock_clustering_label_input = keras.Input(shape=(1,), name='stock_clustering_label')
    stock_clustering_label_embedded = keras.layers.Embedding(stock_clustering_label_orig_dim, stock_clustering_label_emb_dim, input_length=1, name='stock_clustering_label_embedding')(stock_clustering_label_input)
    stock_clustering_label_flattened = keras.layers.Flatten()(stock_clustering_label_embedded)
    raw_input_list.append(stock_clustering_label_input)
    concat_input_list.append(stock_clustering_label_flattened)
    
#     # time clustering embedding
#     time_clustering_label_input = keras.Input(shape=(1,), name='time_clustering_label')
#     time_clustering_label_embedded = keras.layers.Embedding(time_clustering_label_orig_dim, time_clustering_label_emb_dim, input_length=1, name='time_clustering_label_embedding')(time_clustering_label_input)
#     time_clustering_label_flattened = keras.layers.Flatten()(time_clustering_label_embedded)
#     raw_input_list.append(time_clustering_label_input)
#     concat_input_list.append(time_clustering_label_flattened)
    
    # concatencate all input layers
    x = keras.layers.Concatenate()(concat_input_list)
    
    # Add one or more hidden layers
    for n_hidden in hidden_units:
        x = keras.layers.Dense(n_hidden, activation='swish')(x)

    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(x)
    
    model = keras.Model(inputs=raw_input_list, outputs=out)
    model.compile(keras.optimizers.Adam(learning_rate=0.006), loss=root_mean_squared_per_error)
    return model

In [31]:
%%time
hidden_units_list = [(240,200,160,120,80,40,20), (220,180,140,100,50,25), (200,150,100,50,25), (180,120,60,30)]
# hidden_units_list = [(4,2), (4,2), (4,2), (4,2)]
# hidden_units_list = [(128,64,32)]
stock_id_orig_dim, stock_id_emb_dim = 112, 24
stock_clustering_label_orig_dim, stock_clustering_label_emb_dim = 6, 4
# time_clustering_label_orig_dim, time_clustering_label_emb_dim = 8, 4
numerical_feats = [c for c in train_nn if c not in train_key_cols and 'clustering_label' not in c]
train_nn_stock_id = train_nn['stock_id']
train_nn_stock_clustering_label = train_nn['stock_clustering_label']
# train_nn_time_clustering_label = train_nn['time_clustering_label']

def train_and_eval_nn():
    oof_pred_nn_list = []
    test_pred_nn_list = []
    for h in range(len(hidden_units_list)):
        # initialize predictions and scores
        oof_pred_nn = np.zeros(train_nn.shape[0])
        fold_scores = []
        test_pred_nn = []
        for fold in range(N_FOLD):
            if INFERENCE==False:
                print(f'Training fold {fold}...')
                # train-test split
                indexes = np.arange(N_FOLD).astype(int)    
                indexes = np.delete(indexes, obj=fold, axis=0) 
                indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]
                trn_idx = train_nn[train_nn.time_id.isin(indexes)].index.tolist()
                val_idx = train_nn[train_nn.time_id.isin(values[fold])].index.tolist()
                X_train = train_nn.iloc[trn_idx,:][numerical_feats + ['stock_id','stock_clustering_label']]
                y_train = train_nn.iloc[trn_idx,:]['target']
                X_valid = train_nn.iloc[val_idx,:][numerical_feats + ['stock_id','stock_clustering_label']]
                y_valid = train_nn.iloc[val_idx,:]['target']

                # define numerical and categorical data for train set
                X_train_num = X_train[numerical_feats].values
                X_train_stock_id = X_train['stock_id']
                X_train_stock_clustering_label = X_train['stock_clustering_label']
#                 X_train_time_clustering_label = X_train['time_clustering_label']

                # define numerical and categorical data for validation set
                X_valid_num = X_valid[numerical_feats].values
                X_valid_stock_id = X_valid['stock_id']
                X_valid_stock_clustering_label = X_valid['stock_clustering_label']
#                 X_valid_time_clustering_label = X_valid['time_clustering_label']

                # model training
                model = create_nn_model(hidden_units=hidden_units_list[h])
                model.fit([X_train_num, X_train_stock_id, X_train_stock_clustering_label], 
                          y_train,               
                          batch_size=2048,
                          epochs=NN_EPOCH,
                          validation_data=([X_valid_num, X_valid_stock_id, X_valid_stock_clustering_label], y_valid),
                          callbacks=[es, plateau],
                          validation_batch_size=len(y_valid),
                          shuffle=True,
                          verbose=1)

                # validation result
                val_pred = model.predict([X_valid_num, X_valid_stock_id, X_valid_stock_clustering_label]).reshape(1,-1)[0]
                oof_pred_nn[val_idx] = val_pred
                score = round(rmspe(y_valid, val_pred),5)
                fold_scores.append(score)
                print('Fold {}: {}'.format(fold, score))

                # test data prediction
                test_pred_nn.append(model.predict([test_nn[numerical_feats].values, test_nn['stock_id'], test_nn['stock_clustering_label']]).reshape(1,-1)[0].clip(0,1e10))
                # save model
                model.save(f'nn_layer{h}_fold{fold}')
            
            elif INFERENCE==True:
                print(f'Inferring layer {h} fold {fold}')
                path = os.path.join(BASE_MODEL_PATH, f'nn_layer{h}_fold{fold}')
                model = tf.keras.models.load_model(path, compile=False)
                test_pred_nn.append(model.predict([test_nn[numerical_feats].values, test_nn['stock_id'], test_nn['stock_clustering_label']]).reshape(1,-1)[0].clip(0,1e10))
                
            tf.keras.backend.clear_session()
            gc.collect()
    
        # check OOF data quality
        print('Hidden units is ', hidden_units_list[h])
        print(f'Individual folds score is', fold_scores)
        oof_pred_nn_list.append(oof_pred_nn)
        test_pred_nn_list.append(test_pred_nn)
    return oof_pred_nn_list, test_pred_nn_list

oof_pred_nn_list, test_pred_nn_list = train_and_eval_nn()

del train_nn
gc.collect()

Inferring layer 0 fold 0
Inferring layer 0 fold 1
Inferring layer 0 fold 2
Inferring layer 0 fold 3
Inferring layer 0 fold 4
Hidden units is  (240, 200, 160, 120, 80, 40, 20)
Individual folds score is []
Inferring layer 1 fold 0
Inferring layer 1 fold 1
Inferring layer 1 fold 2
Inferring layer 1 fold 3
Inferring layer 1 fold 4
Hidden units is  (220, 180, 140, 100, 50, 25)
Individual folds score is []
Inferring layer 2 fold 0
Inferring layer 2 fold 1
Inferring layer 2 fold 2
Inferring layer 2 fold 3
Inferring layer 2 fold 4
Hidden units is  (200, 150, 100, 50, 25)
Individual folds score is []
Inferring layer 3 fold 0
Inferring layer 3 fold 1
Inferring layer 3 fold 2
Inferring layer 3 fold 3
Inferring layer 3 fold 4
Hidden units is  (180, 120, 60, 30)
Individual folds score is []
CPU times: user 21.1 s, sys: 1 s, total: 22.1 s
Wall time: 26.8 s


15462

# TabNet

In [32]:
if INFERENCE==False:
    train_tbn = pd.read_feather(os.path.join(FE_PATH, 'train_tbn.f'))
    test_tbn = pd.read_feather(os.path.join(FE_PATH, 'test_tbn.f'))

In [33]:
!pip -q install ../input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.matlib

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

from scipy import stats
from scipy.stats import norm
from joblib import Parallel, delayed

import shutil
import glob

from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
import psutil
print(psutil.cpu_count())
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

2
Mon Sep 27 16:22:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    33W / 250W |  16059MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----

In [34]:
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

class RMSPE(Metric):
    def __init__(self):
        self._name = "rmspe"
        self._maximize = False
    def __call__(self, y_true, y_score):
        return np.sqrt(np.mean(np.square((y_true - y_score) / y_true)))

def RMSPELoss(y_pred, y_true):
    return torch.sqrt(torch.mean( ((y_true - y_pred) / y_true) ** 2 )).clone()

In [35]:
# identify categorical and numerical columns
correlated_features = pickle.load(open(os.path.join('/kaggle/input/volatility-correlated-features', f'correlated_features.p'), 'rb'))
cat_cols = ['stock_id','time_clustering_label']
num_cols = [c for c in train_tbn if c not in ['stock_id','time_id','row_id','target'] and 'clustering_label' not in c and c not in correlated_features]
tabnet_feats = [c for c in train_tbn if c in cat_cols + num_cols]

# define categorical features index and dimentions for Tabnet params
cat_idxs = [tabnet_feats.index(c) for c in cat_cols]
cat_dims = [112, 8]
cat_emb_dim = [24, 4]

tabnet_params = dict(
    cat_idxs = cat_idxs,
    cat_dims = cat_dims,
    cat_emb_dim = cat_emb_dim,
    n_d = 24,
    n_a = 24,
    n_steps = 1,
    gamma = 2.0,
    n_independent = 2,
    n_shared = 2,
    lambda_sparse = 2.166158737727093e-06,
    mask_type = "entmax",
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    scheduler_params=dict(mode="min", patience=3, min_lr=1e-5, factor=0.5),
    seed = SEED,
    verbose = 10
)

In [36]:
%%time

import os
import zipfile
def zip_directory(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, mode='w') as zipf:
        len_dir_path = len(folder_path)
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, file_path[len_dir_path:])
    return
                
def train_and_eval_tabnet():
    kfold = GroupKFold(n_splits=N_FOLD)
    oof_pred_tbn = np.zeros(train_tbn.shape[0])
    test_pred_tbn = []
    for fold, (trn_idx, val_idx) in enumerate(kfold.split(train_tbn, train_tbn.target, train_tbn.time_id)):
        if INFERENCE==False:
            print(f'Training fold {fold}......')
            X_train, X_val = train_tbn[tabnet_feats].iloc[trn_idx].values, train_tbn[tabnet_feats].iloc[val_idx].values
            y_train, y_val = train_tbn.target.iloc[trn_idx].values.reshape(-1,1), train_tbn.target.iloc[val_idx].values.reshape(-1,1)
            
            model = TabNetRegressor(**tabnet_params)
            model.fit(
              X_train, y_train,
              eval_set=[(X_val, y_val)],
              max_epochs = TABNET_EPOCH,
              patience = 10,
              batch_size = 1024, 
              virtual_batch_size = 128,
              num_workers = 0,
              drop_last = False,
              eval_metric=[RMSPE],
              loss_fn=RMSPELoss
              )
            # saving model
            saving_path_name = f"TabNet_fold{fold}"
            saved_filepath = model.save_model(saving_path_name)
            # predictions
            oof_pred = model.predict(X_val).flatten()
            oof_pred_tbn[val_idx] = oof_pred
            val_rmspe = rmspe(y_val.flatten(), oof_pred)
            print(f'TabNet fold {fold} RMSPE is {val_rmspe}')
            test_pred_tbn.append(model.predict(test_tbn[tabnet_feats].values).flatten())

        elif INFERENCE==True:
            input_path = os.path.join(BASE_MODEL_PATH, f'TabNet_fold{fold}')
            output_filename = f'TabNet_fold{fold}.zip'
            zip_directory(input_path, output_filename)
            model = TabNetRegressor()
            model.load_model(output_filename)
            test_pred_tbn.append(model.predict(test_tbn[tabnet_feats].values).flatten())

    return oof_pred_tbn, test_pred_tbn

oof_pred_tbn, test_pred_tbn = train_and_eval_tabnet()
print(f'OOF score across folds: {rmspe(y, oof_pred_tbn)}')

del train_tbn
gc.collect()

Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
OOF score across folds: 1.0
CPU times: user 616 ms, sys: 72.8 ms, total: 689 ms
Wall time: 923 ms


0

# Ensembling

In [37]:
# combining oof set
train_meta = pd.DataFrame(np.column_stack([oof_pred_lgb] + [oof for oof in oof_pred_nn_list] + [oof_pred_tbn]),
                                columns=['lgb'] + [f'nn_layer{i}' for i in range(len(hidden_units_list))] + ['tabnet'])
train_meta = pd.concat([train_keys, train_meta], axis=1).reset_index(drop=True)
    
# combining test set    
test_pred_lgb = pd.DataFrame(np.mean(np.column_stack(test_pred_lgb), axis=1), columns=['lgb'])
test_pred_nn = [pd.DataFrame(np.mean(np.column_stack(test_pred_nn_list[i]), axis=1), columns=[f'nn_layer{i}']) for i in range(len(hidden_units_list))]
test_pred_tbn = pd.DataFrame(np.mean(np.column_stack(test_pred_tbn), axis=1), columns=['tabnet'])
test_meta = pd.concat([test_keys] + [test_pred_lgb] + test_pred_nn + [test_pred_tbn], axis=1).reset_index(drop=True)
print(f'Shape of meta test is {test_meta.shape}')

Shape of meta test is (3, 9)


In [38]:
'''
Getting predictions of correlated stocks
'''
# correlation based on realized volatility (prediction target)
def get_sxt_corr_stock_mapping(data, metric, n_top, log_transform, show_distance):
    n_top = min(len(data.stock_id.unique())-1, n_top)
    # calculate correlations
    if log_transform==False:
        corr = pd.pivot_table(data, values=metric, index='time_id', columns='stock_id', aggfunc=np.sum).corr()
    elif log_transform==True:
        corr = np.log(pd.pivot_table(data, values=metric, index='time_id', columns='stock_id', aggfunc=np.sum)).corr()
    # compile mapping table
    mapping = []
    for stock_id in corr.columns:
        df = pd.DataFrame({'nearest_stocks':corr[stock_id].sort_values(ascending=False)[1:n_top+1].index.tolist()})
        if show_distance==True:
            df['nearest_stocks_corr'] = corr[stock_id].sort_values(ascending=False)[1:n_top+1].tolist()
        df['stock_id'] = stock_id
        mapping.append(df)
    mapping = pd.concat(mapping, axis=0).reset_index(drop=True)
    return mapping

# generate mapping table
target = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/train.csv')
corr_stock_mapping = get_sxt_corr_stock_mapping(data=target, metric='target', n_top=5, log_transform=True, show_distance=False)

# cross join with time_id
train_id_list = train_meta[['stock_id','time_id']].drop_duplicates()
corr_stock_mapping_train = pd.merge(corr_stock_mapping, train_id_list, how='inner',on='stock_id')[['stock_id','time_id','nearest_stocks']]
test_id_list = test_meta[['stock_id','time_id']].drop_duplicates()
corr_stock_mapping_test = pd.merge(corr_stock_mapping, test_id_list, how='inner',on='stock_id')[['stock_id','time_id','nearest_stocks']]

# calculate mean predictions of correlated stocks
corr_stock_pred_train = pd.merge(corr_stock_mapping_train, 
                                 train_meta.drop(['row_id','target'], axis=1).rename(columns={'stock_id':'nearest_stocks'}), 
                                 how='inner', 
                                 on=['nearest_stocks','time_id']).\
                        groupby(['stock_id','time_id'])[['lgb','nn_layer0','nn_layer1','nn_layer2','nn_layer3','tabnet']].\
                        mean().\
                        reset_index()
corr_stock_pred_train.columns = ['stock_id','time_id'] + [f'{c}_corr' for c in ['lgb','nn_layer0','nn_layer1','nn_layer2','nn_layer3','tabnet']]
print(f'Shape of train correlated stocks mean prediciton is {corr_stock_pred_train.shape}')
corr_stock_pred_test = pd.merge(corr_stock_mapping_test, 
                                 test_meta.drop(['row_id'], axis=1).rename(columns={'stock_id':'nearest_stocks'}), 
                                 how='inner', 
                                 on=['nearest_stocks','time_id']).\
                        groupby(['stock_id','time_id'])[['lgb','nn_layer0','nn_layer1','nn_layer2','nn_layer3','tabnet']].\
                        mean().\
                        reset_index()
corr_stock_pred_test.columns = ['stock_id','time_id'] + [f'{c}_corr' for c in ['lgb','nn_layer0','nn_layer1','nn_layer2','nn_layer3','tabnet']]
print(f'Shape of test correlated stocks mean prediciton is {corr_stock_pred_test.shape}')

# add to prediction table
train_meta = pd.merge(train_meta, corr_stock_pred_train, how='left', on=['stock_id','time_id'])
test_meta = pd.merge(test_meta, corr_stock_pred_test, how='left', on=['stock_id','time_id'])
# fillna (in case)
train_meta = train_meta.fillna(train_meta.mean()).fillna(0)
test_meta = test_meta.fillna(test_meta.mean()).fillna(0)
# release memory
del corr_stock_mapping, corr_stock_mapping_train, corr_stock_mapping_test, corr_stock_pred_train, corr_stock_pred_test
gc.collect()
# save data
if INFERENCE==False:
    train_meta.to_csv('train_meta.csv', index=False)
    test_meta.to_csv('test_meta.csv', index=False)

Shape of train correlated stocks mean prediciton is (428932, 8)
Shape of test correlated stocks mean prediciton is (0, 8)


### Ensemble method 1: Random Forest

In [39]:
%%time
from sklearn.ensemble import RandomForestRegressor

rf_params = dict(n_estimators = 80,
                max_depth = 9,
                min_samples_split = 14,
                min_samples_leaf = 32,
                max_features = 'sqrt',
                max_samples = 0.45,
                criterion='mse',
                random_state=SEED)

def train_and_eval_meta_rf():
    if INFERENCE==False:
        # model training
        train_weights = 1 / np.square(train_meta.target)
        model = RandomForestRegressor(**rf_params)
        model.fit(X=train_meta[[c for c in train_meta if c not in train_key_cols]],
                  y=train_meta.target,
                  sample_weight=train_weights)
        pickle.dump(model, open(f'meta_rf.p', 'wb'))
    else:
        model = pickle.load(open(os.path.join(BASE_MODEL_PATH, f'meta_rf.p'), 'rb'))
    # predictions
    train_pred_meta = model.predict(train_meta[[c for c in train_meta if c not in train_key_cols]])
    test_pred_meta = model.predict(test_meta[[c for c in test_meta if c not in train_key_cols]])
    return test_pred_meta, train_pred_meta

# test_pred_meta, train_pred_meta = train_and_eval_meta_rf()

CPU times: user 18 µs, sys: 0 ns, total: 18 µs
Wall time: 23.1 µs


### Ensemble method 2: Linear Regression (OLS)

In [40]:
%%time
from sklearn.linear_model import LinearRegression
def train_and_eval_meta_ols():
    if INFERENCE==False:
        # model training
        train_weights = 1 / np.square(train_meta.target)
        model = LinearRegression(fit_intercept=False)
        model.fit(X=train_meta[[c for c in train_meta if c not in train_key_cols]],
                  y=train_meta.target,
                  sample_weight=train_weights)
        print(f'Fitted coefficients are {model.coef_}')
        pickle.dump(model, open(f'meta_ols.p', 'wb'))
    else:
        model = pickle.load(open(os.path.join(BASE_MODEL_PATH, f'meta_ols.p'), 'rb'))
    # predictions
    train_pred_meta = model.predict(train_meta[[c for c in train_meta if c not in train_key_cols]])
    test_pred_meta = model.predict(test_meta[[c for c in test_meta if c not in train_key_cols]])
    return test_pred_meta, train_pred_meta

# test_pred_meta, train_pred_meta = train_and_eval_meta_ols()

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 17.4 µs


### Ensemble method 3: Equal Weight

In [41]:
%%time
def equal_weight_meta():
    nn_cols = [c for c in test_meta if 'nn' in c and c not in train_key_cols]    
    train_pred_meta = (train_meta[nn_cols].mean(axis=1) + train_meta['lgb'] + train_meta['tabnet']) / 3
    test_pred_meta = (test_meta[nn_cols].mean(axis=1) + test_meta['lgb'] + test_meta['tabnet']) / 3     
    return test_pred_meta, train_pred_meta

# test_pred_meta, train_pred_meta = equal_weight_meta()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


### Ensemble method 4: Weighted by CV score

In [42]:
%%time
def cv_score_weighting():
    if INFERENCE==False:
        # initialize cv score tables
        base_cv = []
        nn_cv = []
        # calcualte OOF score of each model
        base_cv.append(('lgb', rmspe(train_keys['target'], oof_pred_lgb)))
        for i in range(len(hidden_units_list)):
            nn_cv.append((f'nn_layer{i}', rmspe(train_keys['target'], oof_pred_nn_list[i])))
        base_cv.append(('tabnet', rmspe(train_keys['target'], oof_pred_tbn)))
        # transform list to table
        base_cv = pd.DataFrame(base_cv, columns=['model','cv'])
        nn_cv = pd.DataFrame(nn_cv, columns=['model','cv'])
        # define contrast parameter
        k_nn = 0.015
        k_base = 0.2
        # calculate NN weights
        nn_cv['imp'] = np.exp((1/k_nn) * 1 / nn_cv['cv'])
        nn_cv['weight'] = nn_cv['imp'] / np.sum(nn_cv['imp'])
        nn_avg_cv = pd.DataFrame({'model':['nn'], 'cv':[np.sum(np.multiply(nn_cv.cv, nn_cv.weight))]})
        # calculate base model weight (including the overall NN)
        base_cv = pd.concat([base_cv, nn_avg_cv], axis=0).reset_index(drop=True)
        base_cv['imp'] = np.exp((1/k_base) * 1 / base_cv['cv'])
        base_cv['weight'] = base_cv['imp'] / np.sum(base_cv['imp'])
        # derive final weight for all models
        nn_cv['weight'] = nn_cv['weight'] * float(base_cv[base_cv.model=='nn']['weight'])
        nn_cv = nn_cv[['model','weight']]
        base_cv = base_cv[base_cv.model!='nn'][['model','weight']]
        base_cv = pd.concat([base_cv, nn_cv], axis=0).reset_index(drop=True)
        model_weights = dict(base_cv.to_records(index=False))
        pickle.dump(model_weights, open(f'model_weights.p', 'wb'))
    else:
        model_weights = pickle.load(open(os.path.join(BASE_MODEL_PATH, f'model_weights.p'), 'rb'))
    # make predictions
    train_pred_meta = sum([train_meta[m] * model_weights[m] for m in model_weights])
    test_pred_meta = sum([test_meta[m] * model_weights[m] for m in model_weights])  
    return test_pred_meta, train_pred_meta

# test_pred_meta, train_pred_meta = cv_score_weighting()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


### Ensemble method 5: ElasticNet

In [43]:
%%time
from sklearn.linear_model import ElasticNet
def train_and_eval_meta_elasticnet():
    if INFERENCE==False:
        # model training
        train_weights = 1 / np.square(train_meta.target)
        model = ElasticNet(fit_intercept=False, alpha=1e-10, l1_ratio=0, positive=False, random_state=SEED, max_iter=10000)
        model.fit(X=train_meta[[c for c in train_meta if c not in train_key_cols]],
                  y=train_meta.target,
                  sample_weight=train_weights)
        print(f'Fitted coefficients are {model.coef_}')
        pickle.dump(model, open(f'meta_elasticnet.p', 'wb'))
    else:
        model = pickle.load(open(os.path.join(BASE_MODEL_PATH, f'meta_elasticnet.p'), 'rb'))
    # predictions
    train_pred_meta = model.predict(train_meta[[c for c in train_meta if c not in train_key_cols]])
    test_pred_meta = model.predict(test_meta[[c for c in test_meta if c not in train_key_cols]])
    return test_pred_meta, train_pred_meta

test_pred_meta, train_pred_meta = train_and_eval_meta_elasticnet()

CPU times: user 26.5 ms, sys: 940 µs, total: 27.4 ms
Wall time: 33.9 ms


# Clipping

In [44]:
# clipping
target = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/train.csv')
min_target, max_target = target.target.min(), target.target.max()
# min_target, max_target = 0, target.target.max()
print(min_target, max_target)

oof_pred_lgb = np.clip(oof_pred_lgb, min_target, max_target)
for i in range(len(hidden_units_list)):
    oof_pred_nn_list[i] = np.clip(oof_pred_nn_list[i], min_target, max_target)
oof_pred_tbn = np.clip(oof_pred_tbn, min_target, max_target)
train_pred_meta = np.clip(train_pred_meta, min_target, max_target)
test_pred_meta = np.clip(test_pred_meta, min_target, max_target)

0.000105263 0.07032062


# Final score and submission

In [45]:
# evaluation
if INFERENCE==False:
    # LightGBM
    lgb_cv_score = round(rmspe(train_keys['target'], oof_pred_lgb), 5)
    print(f'LGBM CV score is {lgb_cv_score}')
    # NN
    for i in range(len(hidden_units_list)):
        cv_score = round(rmspe(train_keys['target'], oof_pred_nn_list[i]), 5)
        print(f'NN{i} CV score is {cv_score}')
    # TabNet
    tbn_cv_score = round(rmspe(train_keys['target'], oof_pred_tbn), 5)
    print(f'TabNet CV score is {tbn_cv_score}')
    # Ensembled
    meta_cv_score = round(rmspe(train_keys['target'], train_pred_meta), 5)
    print(f'Meta Model CV score is {meta_cv_score}')


# submission
test['target'] = test_pred_meta
test = test[['row_id', 'target']].reset_index(drop=True)
test.to_csv('submission.csv', index=False)
display(test.head())

,row_id,target
0,0-4,0.001644
1,0-32,0.001659
2,0-34,0.001659
